In [1]:
!nvidia-smi

Tue Feb 17 17:52:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
%%writefile start.cu
#include <stdio.h>

__global__ void matrixAdd(){
  printf("Hello World from %d \n", threadIdx.x);
}

int main()
{
  int Nx = 12;
  int Ny = 6;

  dim3  threadsPerBlock(4, 3);
  dim3  numBlocks(Nx/threadsPerBlock.x, Ny/threadsPerBlock.y);

  matrixAdd <<<2, 4>>>();
  cudaDeviceSynchronize();


}

Overwriting start.cu


In [25]:
!nvcc -arch=sm_75 start.cu -o start

In [26]:
!./start

Hello World from 0 
Hello World from 1 
Hello World from 2 
Hello World from 3 
Hello World from 0 
Hello World from 1 
Hello World from 2 
Hello World from 3 


Q4

In [6]:
%%writefile q4.cu
#include <stdio.h>

__global__ void helloWorld(){
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    printf("Hello World from GPU thread = %d\n", i);
}

int main()
{
  printf("Launching Hello World Kernel on GPU\n");
  helloWorld<<<1, 8>>>();
  cudaDeviceSynchronize();
  printf("Kernel Execution on GPU complete");
  return 0;
}

Overwriting q3.cu


In [7]:
!nvcc -arch=sm_75 q4.cu -o q4

In [8]:
!./q3

Launching Hello World Kernel on GPU
Hello World from GPU thread = 0
Hello World from GPU thread = 1
Hello World from GPU thread = 2
Hello World from GPU thread = 3
Hello World from GPU thread = 4
Hello World from GPU thread = 5
Hello World from GPU thread = 6
Hello World from GPU thread = 7
Kernel Execution on GPU complete

Q5

In [31]:
%%writefile q5.cu
#include <stdio.h>

__global__ void mykernel(int arr[])
{
  for (int i = 0; i < 5; i++)
  {
    printf("%d\n", arr[i]);
  }
}

int main()
{
  int a[5] = {1, 2, 3, 4, 5};
  int *d_a;
  int size = sizeof(a);
  cudaMalloc((void **) &d_a, size);

  cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
  mykernel<<<1,1>>>(d_a);
  printf("Result from GPU\n");
  cudaDeviceSynchronize();

  cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
  printf("Result from CPU\n");
  for (int i = 0; i < 5; i++)
  {
    printf("%d\n", a[i]);
  }
}


Overwriting q5.cu


In [32]:
!nvcc -arch=sm_75 q5.cu -o q5

In [33]:
!./q5

Result from GPU
1
2
3
4
5
Result from CPU
1
2
3
4
5
